In [15]:
import configparser
import os.path as op

CP = configparser.ConfigParser()
CP.read_file(open(op.join(op.expanduser('~'), '.aws', 'credentials')))
CP.sections()
aws_access_key = CP.get('hcp', 'AWS_ACCESS_KEY_ID')
aws_secret_key = CP.get('hcp', 'AWS_SECRET_ACCESS_KEY')

def attach_keys(arr):
    return [(e, aws_access_key, aws_secret_key) for e in arr]

In [33]:
def afq_hcp_api(args):
    import logging
    import os.path as op
    import s3fs

    from AFQ.data import fetch_hcp
    import AFQ.api as api
    
    logging.basicConfig(level=logging.INFO)
    log = logging.getLogger(__name__)  
    
    fs = s3fs.S3FileSystem()
    
    subject = args[0]
    aws_access_key = args[1]
    aws_secret_key = args[2]

    fetch_hcp([subject], profile_name=False,
              aws_access_key_id=aws_access_key,
              aws_secret_access_key=aws_secret_key)
    derivatives = op.join(op.expanduser('~'), "AFQ_data/HCP/derivatives");
    
    myafq = api.AFQ(op.join(derivatives, "dmriprep"),
                    b0_threshold=50,
                    sub_prefix="sub-",
                    tracking_params = {"odf_model": "DKI"},
                    #virtual_frame_buffer=True,
                    scalars=["dti_fa", "dti_md", "dki_fa", "dki_md"])
    
    myafq.export_registered_b0()
#     myafq.export_bundles()
#     myafq.get_tract_profiles()
#     myafq.export_bundle_gif()
#     myafq.export_ROI_gifs()
    fs.put(op.join(derivatives, "afq"), "profile-hcp", recursive=True)


In [34]:
subjects_num = [
    101208,
    102109,
    102614,
    102715,
    103010,
    103212,
    106824,
    108020,
    111211,
    113316,
    113417,
    114116,
    115724,
    116423,
    117021,
    117728,
    118831,
    119025,
    120010,
    120414,
    120818,
    122418,
    123723,
    125222,
    125424,
    126426,
    127226,
    127731,
    127832,
    130114,
    130518,
    130720,
    134627,
    135124,
    135629,
    136126,
    136328,
    136530,
    136631,
    137431,
    137532,
    138130,
    138332,
    139435,
    143224,
    143830,
    144933,
    145632,
    146735,
    146836
]
subjects = [str(i) for i in subjects_num] 
args = attach_keys(subjects)
#afq_hcp_api(args[0])

In [35]:
import cloudknot as ck
ck.set_region('us-east-1')

In [36]:
knot = ck.Knot(name='hcp_api-64gb-20708-2',
              func=afq_hcp_api,
              base_image='libglxvfb:1',
              image_github_installs="https://github.com/yeatmanlab/pyAFQ.git",
              pars_policies=('AmazonS3FullAccess',),
              resource_type="SPOT",
              bid_percentage=100,
              memory=64000)

In [37]:
result_futures = knot.map(attach_keys(['108020', '111211']))

In [39]:
knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
65667b23-e42d-46ac-a4df-4f3d47aeb289        hcp-api-64gb-20708-2-0        SUCCEEDED


In [24]:
j0 = knot.jobs[0]
j0.status

{'status': 'FAILED',
 'statusReason': 'Array Child Job failed',
 'attempts': [],
 'arrayProperties': {'statusSummary': {'STARTING': 0,
   'FAILED': 2,
   'RUNNING': 0,
   'SUCCEEDED': 0,
   'RUNNABLE': 0,
   'SUBMITTED': 0,
   'PENDING': 0},
  'size': 2}}

In [40]:
knot.clobber(clobber_pars=True, clobber_repo=True, clobber_image=True)